<a href="https://colab.research.google.com/github/INT-Group-9/ColabNotebook/blob/clara/cifa10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#literally copy pasted code :(
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128, 64, 3)
        self.fc1 = nn.Linear(64*6*6, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*6*6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.073
[1,  4000] loss: 1.664
[1,  6000] loss: 1.508
[1,  8000] loss: 1.401
[1, 10000] loss: 1.316
[1, 12000] loss: 1.234
[2,  2000] loss: 1.123
[2,  4000] loss: 1.119
[2,  6000] loss: 1.067
[2,  8000] loss: 1.034
[2, 10000] loss: 1.013
[2, 12000] loss: 0.977
[3,  2000] loss: 0.880
[3,  4000] loss: 0.855
[3,  6000] loss: 0.881
[3,  8000] loss: 0.843
[3, 10000] loss: 0.861
[3, 12000] loss: 0.837
[4,  2000] loss: 0.695
[4,  4000] loss: 0.742
[4,  6000] loss: 0.745
[4,  8000] loss: 0.730
[4, 10000] loss: 0.749
[4, 12000] loss: 0.695
[5,  2000] loss: 0.582
[5,  4000] loss: 0.599
[5,  6000] loss: 0.615
[5,  8000] loss: 0.625
[5, 10000] loss: 0.627
[5, 12000] loss: 0.648
[6,  2000] loss: 0.472
[6,  4000] loss: 0.498
[6,  6000] loss: 0.525
[6,  8000] loss: 0.534
[6, 10000] loss: 0.549
[6, 12000] loss: 0.542
[7,  2000] loss: 0.387
[7,  4000] loss: 0.425
[7,  6000] loss: 0.424
[7,  8000] loss: 0.438
[7, 10000] loss: 0.471
[7, 12000] loss: 0.463
[8,  2000] loss: 0.300
[8,  4000] 

In [12]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [7]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [13]:
#outputs = net(images)
#this doesnt work cos i left out the "lets show some images" thing from the tutorial but i accidentally
#copied this in oops

In [9]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:  plane   dog  frog  ship


In [10]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 71 %


In [11]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 73.0 %
Accuracy for class car   is: 81.9 %
Accuracy for class bird  is: 66.0 %
Accuracy for class cat   is: 50.3 %
Accuracy for class deer  is: 66.4 %
Accuracy for class dog   is: 64.0 %
Accuracy for class frog  is: 79.3 %
Accuracy for class horse is: 77.3 %
Accuracy for class ship  is: 80.5 %
Accuracy for class truck is: 79.4 %
